In [1]:
# import
import csv
import math
import random
import sys
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from pystruct.models import ChainCRF
from pystruct.learners import FrankWolfeSSVM

from sklearn.cluster import KMeans

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic,
                                              ExpSineSquared, DotProduct,
                                              ConstantKernel, WhiteKernel)

%matplotlib inline

random.seed

<bound method Random.seed of <random.Random object at 0x102846a18>>

In [2]:
# for floor in range(0,5):
#     for building in range(0,3):
for floor in range(2,3):
    for building in range(0,1):
        
        ## Count the number of data points in building id & floor id
        data_num = 0
        with open("1478167720_9233432_trainingData.csv", newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for row in spamreader:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                data_num += 1
        print(data_num)
        ## if there are no data, continue to next floor 
        if (data_num == 0):
            continue
            
        ## Load data points in
        wifi_loc_time = np.zeros(shape = (data_num, 524))
        i=-1
        with open("1478167720_9233432_trainingData.csv", newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for row in spamreader:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                i = i+1
                if (i > data_num):
                    break
                # wifi
                wifi_loc_time[i-1][:520] = np.array(row[:520])
                # location x, y
                wifi_loc_time[i-1][520:522] = np.array(row[520:522])
                # userID
                wifi_loc_time[i-1][522] = np.array(row[526])
                # time stamp
                wifi_loc_time[i-1][-1] = np.array(row[-1])
        
        ## Sort by time stamp
        wifi_loc_time = wifi_loc_time[wifi_loc_time[:,-1].argsort()]
        
        ## List all users
        user_list = np.unique(wifi_loc_time[:, 522])
        user_num = len(user_list)
        print("On floor ", floor, " in building ", building, "there are ", user_num, " users.")
        
                                    ## max boundary , grid size, min boundary
        grid_slice = 1
        longitude_list = np.array([max(wifi_loc_time[:, 520]), (max(wifi_loc_time[:, 520])-min(wifi_loc_time[:, 520]))/grid_slice\
                                   , min(wifi_loc_time[:, 520])])
        latitude_list = np.array([max(wifi_loc_time[:, 521]), (max(wifi_loc_time[:, 521])-min(wifi_loc_time[:, 521]))/grid_slice\
                                   , min(wifi_loc_time[:, 521])])
        
        ## set the sequence length of observed walking tracks
        seq_window = 9
        total_batch_num = 0
        
        ## Record each user's number of location logs
        user_walking_num = np.zeros(shape=(user_num), dtype=int)
        for i in range(len(wifi_loc_time[:,522])):
            for user_index in range(user_num):
                if (wifi_loc_time[i, 522] == user_list[user_index]):
                    user_walking_num[user_index] += 1
        
        ## Compute all users' walking sequences and add them as number of batches
        for num in user_walking_num:
            total_batch_num += math.floor(num/seq_window)
        print("total_batch_num: ",total_batch_num)
        
        batch_wifi = np.zeros((total_batch_num, seq_window, 520))
        batch_loc = np.zeros((total_batch_num, seq_window, 2), dtype = int)
        
        ## cumulative graduation [# of batch of user1] + [# of batch of user2] [...]
        batch_num = 0
        iter_batch = 0
        iter_window = 0
        
        for k in range(len(user_list)):
            print("User",k, " walking ", user_walking_num[k])
            
            ## [longitude, latitude, wifi]
            user_fp = np.zeros((user_walking_num[k], 2+520))
            batch_num += math.floor(user_walking_num[k]/seq_window)
            
            ## pick out the corresponding user's data
            for i in range(len(wifi_loc_time[:,522])):
                if (wifi_loc_time[i, 522] == user_list[k]):
                    batch_wifi[iter_batch][iter_window] = np.array(wifi_loc_time[i, :520])
                    batch_loc[iter_batch,iter_window] = np.array(wifi_loc_time[i, 520:522])

                    if (iter_window == (seq_window-1)):
                        iter_window = 0
                        iter_batch = iter_batch + 1
                        if (iter_batch == (batch_num)):
                            break
                    else:
                        iter_window = iter_window + 1
        ## Data preparation finishs
        ## batch_wifi, shape = (total_batch_num, seq_window, 520)
        ## batch_loc, shape = (total_batch_num, seq_window, 2)
        
        ### DQN design
        max_search_steps = 10
        T = max_search_steps
        model = nn.Sequential(
                    nn.Linear(520 + 2 + 1 + 50 + 1, 512),
                    nn.Linear(512, 512),
                    nn.Linear(512, 1)
                )
        loss_fn = nn.MSELoss()
        
        learning_rate = 1e-4
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        
        for k in range(N):
            print("Training k = ", k)
            for i in range(total_batch_num):
                for j in range(seq_window):
                    print("Batch = ", i, j)
                    # initial coordinates and radius
                    # radius_gt = target window
                    radius_gt = 0.5
                    coordinate = np.array([(longitude_list[0]+longitude_list[2])/2. , (latitude_list[0]+latitude_list[2])/2.])
                    radius = max(longitude_list[0]-longitude_list[2], latitude_list[0]-latitude_list[2])/2. + radius_gt
                    
                    # initial history, 5n vector
                    history = np.zeros(shape=(5*max_search_steps,), dtype=int)
                    
                    # initial state: RSSI (520), coordinate (2), radius (1), history (50)
                    state = (batch_wifi[i][j], coordinate.copy(), radius, history.copy())
                    
                    # initial memory
                    memory = []
                    
                    # Start a searching round
                    # eps
                    eps = 1.
                    action = 0
                    alpha = 0.7
                    delta = 0.5
                    IoW_last_time = 0
                    Reward = 0.
                    for t in range(T):
                        print("search round ", t)
                        # Select an action
                        if (random.random() < eps):
                            action = random.randint(0,4)
                        else:
                            # max Q ::NOTE::
                            input_layer_np = np.concatenate((batch_wifi[i][j], coordinate.copy(), radius, history.copy(), action), axis=0)
                            input_layer = torch.from_numpy(input_layer_np)
                            output_layer = model(input_layer)
                            action = output_layer.item()
                        
                        ### IoW cross section calculation
                        cross_list_0 = [(batch_loc[i,j,0]-radius_gt, 0), (batch_loc[i,j,0]+radius_gt, 0)]
                        cross_list_1 = [(batch_loc[i,j,1]-radius_gt, 0), (batch_loc[i,j,1]+radius_gt, 0)]
                        
                        ### 1. New Center
                        ## 0 -> "Up Left"
                        ## 1 -> "Up Right"
                        ## 2 -> "Down Left"
                        ## 3 -> "Down Right"
                        ## 4 -> "Center"
                        if (action == 0):
                            coordinate[0] -= radius/2.
                            coordinate[1] += radius/2.
                        elif (action == 1):
                            coordinate[0] += radius/2.
                            coordinate[1] -= radius/2.
                        elif (action == 2):
                            coordinate[0] -= radius/2.
                            coordinate[1] -= radius/2.
                        elif (action == 3):
                            coordinate[0] += radius/2.
                            coordinate[1] -= radius/2.
                        
                        ### 2. New Radius
                        ## radius_t+1 = alpha * radius_t
                        radius *= alpha
    
                        ### IoW cross section calculation
                        cross_list_0.append((coordinate[0]-radius, 1))
                        cross_list_0.append((coordinate[0]+radius, 1))
                        cross_list_1.append((coordinate[1]-radius, 1))
                        cross_list_1.append((coordinate[1]+radius, 1))
                        
                        cross_list_0.sort(key = lambda x : x[0])
                        cross_list_1.sort(key = lambda x : x[0])
        
                        ### 3. Reward
                        if (cross_list_0[0,1] != cross_list_0[1,1] and cross_list_1[0,1] != cross_list_1[1,1]):
                            IoW_transform = (cross_list_0[2,0] - cross_list_0[1,0]) * (cross_list_1[2,0] - cross_list_1[1,0]) / math.sqrt(radius*2)
                        
                        if (IoW_transform > delta):
                            Reward += 3.0
                        elif (IoW_transform > IoW_last_time):
                            Reward += 1.0
                        else:
                            Reward -= 3.0
                        
                        ### 4. State
                        current_state = (batch_wifi[i][j], coordinate.copy(), radius, history.copy())
                        
                        ### 5. History
                        history[t*5 + action] = 1
                        
                        ### 6. Memory
                        memory.append((state.copy(), action, Reward, current_state.copy()))
                        state = current_state
                        
                        IoW_last_time = IoW_transform
                        
                        ### Set target value
                        input_layer_np = np.concatenate((batch_wifi[i][j], coordinate.copy(), radius, history.copy(), action), axis=0)
                        input_layer = torch.from_numpy(input_layer_np)
                        output_layer = model(input_layer)
                        action_sgd = output_layer.item()
                        coordinate_sgd = coordinate.copy()
                        radius_sgd = radius
                        Reward_sgd = Reward

                        cross_list_0 = [(batch_loc[i,j,0]-radius_gt, 0), (batch_loc[i,j,0]+radius_gt, 0)]
                        cross_list_1 = [(batch_loc[i,j,1]-radius_gt, 0), (batch_loc[i,j,1]+radius_gt, 0)]
                        
                        
                        if (action_sgd == 0):
                            coordinate_sgd[0] -= radius/2.
                            coordinate_sgd[1] += radius/2.
                        elif (action_sgd == 1):
                            coordinate_sgd[0] += radius/2.
                            coordinate_sgd[1] -= radius/2.
                        elif (action_sgd == 2):
                            coordinate_sgd[0] -= radius/2.
                            coordinate_sgd[1] -= radius/2.
                        elif (action_sgd == 3):
                            coordinate_sgd[0] += radius/2.
                            coordinate_sgd[1] -= radius/2.
                        
                        ### 2. New Radius
                        ## radius_t+1 = alpha * radius_t
                        radius_sgd *= alpha
    
                        ### IoW cross section calculation
                        cross_list_0.append((coordinate_sgd[0]-radius_sgd, 1))
                        cross_list_0.append((coordinate_sgd[0]+radius_sgd, 1))
                        cross_list_1.append((coordinate_sgd[1]-radius_sgd, 1))
                        cross_list_1.append((coordinate_sgd[1]+radius_sgd, 1))
                        
                        cross_list_0.sort(key = lambda x : x[0])
                        cross_list_1.sort(key = lambda x : x[0])
        
                        ### 3. Reward
                        if (cross_list_0[0,1] != cross_list_0[1,1] and cross_list_1[0,1] != cross_list_1[1,1]):
                            IoW_transform = (cross_list_0[2,0] - cross_list_0[1,0]) * (cross_list_1[2,0] - cross_list_1[1,0]) / math.sqrt(radius_sgd*2)
                        
                        if (IoW_transform > delta):
                            Reward_sgd += 3.0
                        elif (IoW_transform > IoW_last_time):
                            Reward_sgd += 1.0
                        else:
                            Reward_sgd -= 3.0
                        ############
                        Y = Reward + 0.1 * Reward_sgd
                        loss = loss_fn(Y, Reward)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        
                        eps = eps * 0.995

            
print("End! cong!")       
                    
#         # Testing
#         # extract data
#         print("Validation...")
#         data_num_v = 0
#         with open("1478167721_0345678_validationData.csv", newline='') as v_csvfile:
#             spamreader_v = csv.reader(v_csvfile, delimiter=',')
#             for row in spamreader_v:
#                 if (row[523] == 'BUILDINGID'):
#                     continue
#                 elif (int(row[523]) is not building or int(row[522]) is not floor):
#                     continue
#                 data_num_v += 1

#         print(data_num_v)
#         if (data_num_v == 0):
#             continue
#         i = -1
#         wifi_loc_time_v = np.zeros(shape = (data_num_v, 524))
        
#         with open("1478167721_0345678_validationData.csv", newline='') as v_csvfile:
#             spamreader_v = csv.reader(v_csvfile, delimiter=',')
#             for row in spamreader_v:
#                 if (row[523] == 'BUILDINGID'):
#                     continue
#                 elif (int(row[523]) is not building or int(row[522]) is not floor):
#                     continue
#                 i += 1
#                 #if (i > data_num):
#                 #    break
#                 # wifi
#                 wifi_loc_time_v[i-1][:520] = np.array(row[:520])
#                 # location x, y
#                 wifi_loc_time_v[i-1][520:522] = np.array(row[520:522])
#                 # userID
#                 wifi_loc_time_v[i-1][522] = np.array(row[526])
#                 # time stamp
#                 wifi_loc_time_v[i-1][-1] = np.array(row[-1])
                
#         wifi_loc_time_v = wifi_loc_time_v[wifi_loc_time_v[:,-1].argsort()]  
                
#         batch_num = int(data_num_v/seq_window)
#         print("Validating data ", batch_num)
#         batch_wifi_v = np.zeros((batch_num + 1, seq_window, 520))
#         batch_loc_v = np.zeros((batch_num + 1, seq_window), dtype = int)

#         iter_window = 0
#         iter_batch = 1
        
#         map_distance = 0
#         adjust_grid_num = 0

#         for s in range(len(wifi_loc_time_v)):
            
#             i = int(math.ceil((wifi_loc_time_v[s][520]-longitude_list[2])/longitude_list[1]) - 1)
#             j = int(math.ceil((wifi_loc_time_v[s][521]-latitude_list[2])/latitude_list[1]) - 1)
#             if (i < -1):
#                 i = 0
#             elif (i >= grid_slice): 
#                 i = grid_slice-1
#             if (j < 0):
#                 j = 0
#             elif (j >= grid_slice): 
#                 j = grid_slice-1
#             if (state_tag[i][j] == -1):
#                 adjust_grid_num+=1
#                 min_dist = np.zeros(shape=(state_count, 1))
#                 for x in range(state_count):
#                     min_dist[x,0] = math.sqrt(pow((float(state_map_grid[x][0])+0.5)*longitude_list[1]+longitude_list[2] - wifi_loc_time_v[s][520],2)+\
#                               pow((float(state_map_grid[x][1])+0.5)*latitude_list[1]+latitude_list[2] - wifi_loc_time_v[s][521],2))
#                 map_distance += np.min(min_dist)
#                 state_tag[i][j] = np.argmin(min_dist)
            
#             batch_wifi_v[iter_batch][iter_window] = wifi_loc_time_v[s][:520]
#             batch_loc_v[iter_batch,iter_window] = state_tag[i][j]
#             if (iter_window == (seq_window-1)):
#                 iter_window = 0
#                 iter_batch = iter_batch + 1
#                 if (iter_batch == (batch_num)):
#                     break
#             else:
#                 iter_window = iter_window + 1  
#         print("Grids need adjustment: ", adjust_grid_num)
#         print("Base map distance is ", map_distance/data_num_v)
                
#         model = ChainCRF()
#         ssvm = FrankWolfeSSVM(model=model, C=.1, max_iter=80)            
#         ssvm.fit(batch_wifi, batch_loc)
        
#         pred_batch_v = ssvm.predict(batch_wifi_v)
#         total_error = 0

#         center_longitude_pred = 0
#         center_latitude_pred = 0
#         center_longitude_gt = 0
#         center_latitude_gt = 0
        
#         real_longitude = 0
#         real_latitude = 0

#         iter_test = 0
        
#         window = np.zeros((seq_window))
        
#         gp_total_error = 0
#         gp_window = np.zeros((seq_window))
        
#         for i in range(1,batch_num+1):
#             for j in range(seq_window):
#                 ans = state_map_grid[pred_batch_v[i][j]][2].predict(batch_wifi_v[i][j].reshape(1,520))
                
#                 center_longitude_pred = (float(state_map_grid[pred_batch_v[i][j]][0])+0.5)*longitude_list[1]
#                 center_latitude_pred = (float(state_map_grid[pred_batch_v[i][j]][1])+0.5)*latitude_list[1]
#                 center_longitude_gt = (float(state_map_grid[batch_loc_v[i][j]][0])+0.5)*longitude_list[1]
#                 center_latitude_gt = (float(state_map_grid[batch_loc_v[i][j]][1])+0.5)*latitude_list[1]
                
                
                
                
#                 ans[0][0] += center_longitude_pred + longitude_list[2]
#                 ans[0][1] += center_latitude_pred + latitude_list[2]
#                 real_longitude = wifi_loc_time_v[iter_test][520]
#                 real_latitude = wifi_loc_time_v[iter_test][521]
                
#                 gp_total_error += math.sqrt(pow(ans[0][0] - real_longitude,2)+pow(ans[0][1] - real_latitude,2))
#                 gp_window[j] += math.sqrt(pow(ans[0][0] - real_longitude,2)+pow(ans[0][1] - real_latitude,2))
                
#                 iter_test+=1
                
#                 total_error += math.sqrt(pow(center_longitude_pred - center_longitude_gt,2)+pow(center_latitude_pred - center_latitude_gt,2))
                            
#                 window[j] += math.sqrt(pow(center_longitude_pred - center_longitude_gt,2)+pow(center_latitude_pred - center_latitude_gt,2))
                

#         print("\nbox\nRMSE: ", total_error/(batch_num*seq_window))
#         print("Windows:")
#         for x in range(seq_window):
#             print(" ", x, window[x]/batch_num)
#         print("gp")
#         print("RMSE: ", gp_total_error/(batch_num*seq_window))
#         print("Windows:")
#         for x in range(seq_window):
#             print(" ", x, gp_window[x]/batch_num)
#         print("Finish\n")

1443
On floor  2  in building  0 there are  2  users.


NameError: name 'grid_slice' is not defined